In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [2]:
actionsPath = "actions.txt"
appsPath = "apps.txt"
usersPath = "users.txt"
actionsRDD = sc.textFile(actionsPath)
appsRDD = sc.textFile(appsPath)
usersRDD = sc.textFile(usersPath)
output1 = "outPart1"
output2 = "outPart2"

In [3]:
# User15,App10,2019/01/01-23:01:15,Install
def filterAppIn21(l):
    date = str(l.split(',')[2])
    return date.startswith("2021/")

def mapAppByIDMonthAndTInstallRemove(l):
    appId = str(l.split(',')[1])
    month = str(l.split(',')[2].split('/')[1])
    t = (0, 0)
    if (str(l.split(',')[3]) == "Install"):
        t = (1, 0)
    if (str(l.split(',')[3]) == "Remove"):
        t = (0, 1)
    return ((appId, month), t)

actions21 = actionsRDD\
        .filter(filterAppIn21)\
        .map(mapAppByIDMonthAndTInstallRemove)\
        .reduceByKey(lambda t1, t2: (t1[0]+t2[0], t1[1]+t2[1]))

In [4]:
actions21.collect()

[(('C1app1', '05'), (65, 64)),
 (('C1app1', '06'), (65, 64)),
 (('C1app1', '08'), (65, 64)),
 (('C1app1', '11'), (65, 64)),
 (('C1app1', '12'), (65, 64)),
 (('C1app2', '01'), (10, 99)),
 (('C1app2', '02'), (10, 99)),
 (('C1app2', '03'), (10, 99)),
 (('C1app2', '04'), (10, 99)),
 (('C1app2', '07'), (10, 99)),
 (('C1app2', '09'), (10, 99)),
 (('C1app2', '10'), (10, 99)),
 (('C1app3', '05'), (2, 1)),
 (('C1app3', '06'), (2, 1)),
 (('C1app3', '08'), (2, 1)),
 (('C1app3', '11'), (2, 1)),
 (('C1app3', '12'), (2, 1)),
 (('C1app4', '05'), (0, 3)),
 (('C1app4', '06'), (0, 3)),
 (('C1app4', '08'), (0, 3)),
 (('C1app4', '11'), (0, 3)),
 (('C1app4', '12'), (0, 3)),
 (('C2app2', '01'), (100, 0)),
 (('C2app3', '01'), (100, 0)),
 (('C2app4', '01'), (300, 0)),
 (('C1app1', '01'), (65, 64)),
 (('C1app1', '02'), (65, 64)),
 (('C1app1', '03'), (65, 64)),
 (('C1app1', '04'), (65, 64)),
 (('C1app1', '07'), (65, 64)),
 (('C1app1', '09'), (65, 64)),
 (('C1app1', '10'), (65, 64)),
 (('C1app2', '05'), (10, 99)

In [5]:
def mapactions21AllMonths(elem):
    if(elem[1][0] > elem[1][1]):
        return (elem[0][0], 1)
    else:
        return (elem[0][0], 0)

actions21AllMonths = actions21.map(mapactions21AllMonths)

In [6]:
appAll21MonthsInstallation = actions21AllMonths\
                        .reduceByKey(lambda v1, v2: v1 + v2)\
                        .filter(lambda elem: elem[1] == 12)

In [7]:
appAll21MonthsInstallation.collect()

[('C1app1', 12), ('C1app3', 12)]

In [8]:
app = appsRDD\
    .map(lambda l: (l.split(',')[0], l.split(',')[1]))

In [9]:
res = app.join(appAll21MonthsInstallation)\
    .map(lambda elem: (elem[0], elem[1][0]))

In [10]:
res.collect()

[('C1app3', 'app3'), ('C1app1', 'app1')]

In [11]:
# Part2

In [17]:
# User15,App10,2019/01/01-23:01:15,Install
def filterInstall(l):
    action = str(l.split(',')[3])
    return action == "Install"
    
actionsInstall = actionsRDD\
            .filter(filterInstall)\
            .map(lambda l: ((l.split(',')[1], l.split(',')[0]), l.split(',')[2]))\
            .cache()

actionsInstallAfter2022 = actionsInstall\
                    .filter(lambda elem: elem[1].split("-")[0] > "2021/12/31")\
                    .distinct()

actionsInstallBefore2022 = actionsInstall\
                    .filter(lambda elem: elem[1].split("-")[0] <= "2021/12/31")\
                    .distinct()

In [30]:
installationAfter2022 = actionsInstallAfter2022\
                .subtractByKey(actionsInstallBefore2022)\
                .map(lambda elem: (elem[0][0],elem[0][1]))\
                .distinct()\
                .map(lambda elem: (elem[0], 1))\
                .reduceByKey(lambda v1, v2: v1+v2)
maxCount = installationAfter2022.reduce(lambda t1, t2: (None, max(t1[1], t2[1])))[1]             


In [33]:
res2 = installationAfter2022.filter(lambda elem: elem[1]==maxCount).keys()

In [34]:
res2.collect()

['C2app3', 'C2app4', 'C2app2']